In [1]:
import logging
import yaml
import asyncio

import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel import ATTCS

import SALPY_ATPtg

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Brings up remotes 
attcs = ATTCS()

In [4]:
# attcs.log.setLevel(logging.DEBUG)

In [239]:
salobj.State(attcs.atpneumatics.evt_summaryState.get().summaryState)

<State.ENABLED: 2>

In [297]:
await salobj.set_summary_state(attcs.atpneumatics, salobj.State.STANDBY, timeout=60)

In [ ]:
for c in attcs.components():
    r = getattr(attcs, c.lower())
    state_topic = await r.evt_summaryState.next(flush=False, timeout=5.)
    state = salobj.State(state_topic.summaryState)
    print(f"{c}::{state!r}")

In [ ]:
state_topic = await attcs.atpneumatics.evt_summaryState.next(flush=False, timeout=5.)
state = salobj.State(state_topic.summaryState)
print(f"{attcs.atpneumatics.salinfo.name}::{state!r}")

In [ ]:
await attcs.atpneumatics.cmd_start.start(timeout=300)

In [ ]:
await attcs.atpneumatics.cmd_enable.start(timeout=300)

In [ ]:
await attcs.atpneumatics.cmd_openMasterAirSupply.start(timeout=300)

In [325]:
#await attcs.atpneumatics.cmd_openM1CellVents.start(timeout=10)
await attcs.atpneumatics.cmd_closeM1CellVents.start(timeout=10)

CommandIdAck(cmd_id=1564155809, ack=(ack=303, error=0, result='Ok'))

In [ ]:
await attcs.atpneumatics.cmd_openInstrumentAirValve.start(timeout=300)

In [199]:
await attcs.atpneumatics.cmd_m1OpenAirValve.start()
#await attcs.atpneumatics.cmd_m1CloseAirValve.start()

CommandIdAck(cmd_id=1564481722, ack=(ack=303, error=0, result='Ok'))

In [200]:
#attcs.atpneumatics.cmd_m1SetPressure.set(pressure=0.1)
#attcs.atpneumatics.cmd_m1SetPressure.set(pressure=9.18799152e+04) # 
attcs.atpneumatics.cmd_m1SetPressure.set(pressure=500.0) # 

False

In [201]:
await attcs.atpneumatics.cmd_m1SetPressure.start()

CommandIdAck(cmd_id=1564514560, ack=(ack=303, error=0, result='Ok'))

In [296]:
await attcs.atpneumatics.cmd_closeM1Cover.start()

CommandIdAck(cmd_id=1564252341, ack=(ack=303, error=0, result='Ok'))

# Measure maximum load on cell at a specified altitude

In [6]:
class MeasureM1Pressure():
    
    def __init__(self, attcs):
        
        self.attcs = attcs
        self.elevation = np.array([])
        self.pressure = np.array([])
        self.load_off = np.array([])
        self.max_load = np.array([])
        self.min_load = np.array([])
        
        self.min_allowed_load = 120.
        self.optimum_fraction = 1./3.
        self.tolerance = 0.01
        
    async def get_load(self, n=10):
        
        load_measures = np.zeros(n)
        for i in range(n):
            load = await self.attcs.atpneumatics.tel_loadCell.next(flush=True, timeout=2)
            load_measures[i] = load.cellLoad
            
        return np.mean(load_measures), np.std(load_measures)
    
    async def get_pressure(self, n=10):
        # Averages 10 pressure measurements
        pressure_measures = np.zeros(n)
        for i in range(n):
            topic = await self.attcs.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=3)
            pressure_measures[i] = topic.pressure
            
        #return np.mean(pressure_measures)*u.Pa
        return np.mean(pressure_measures)*u.Pa, np.std(pressure_measures)*u.Pa  

    async def measure_optimum_pressure(self):
        
        attcs = self.attcs
        
        encoders = attcs.atmcs.tel_mountEncoders.get()
        
        await attcs.atpneumatics.cmd_m1CloseAirValve.start()
        await asyncio.sleep(5.)
        
        closed_air_load, closed_air_load_std = await self.get_load()
        
        optimum_load = closed_air_load * self.optimum_fraction

        print(f"Optimum load [N]: {optimum_load}")
        
        await attcs.atpneumatics.cmd_m1OpenAirValve.start()    

        attcs.atpneumatics.cmd_m1SetPressure.set(pressure=0.1)
        self.attcs.atpneumatics.evt_m1SetPressure.flush()
        await attcs.atpneumatics.cmd_m1SetPressure.start()
        # should put a wait in here.. takes a couple seconds to adjust/settle
        try:
            sp = await self.attcs.atpneumatics.evt_m1SetPressure.next(flush=False, timeout=30)
        except asyncio.TimeoutError:
            pass

        min_pressure, min_pressure_std = await self.get_pressure()

        print(f"Minimum pressure [Pa] (mean, std): [{min_pressure}, {min_pressure_std}]")

#         max_pressure = 15.*np.sin(np.radians(encoders.elevationCalculatedAngle))*u.imperial.psi
# Where does this come from? The min pressure here is essentially zero always.
# and what is the 9?
        max_pressure = 9. * np.sin(np.radians(encoders.elevationCalculatedAngle)) * min_pressure

        print(f"Maximum pressure [Pa]: {max_pressure.to(u.Pa)}")

        print(f"Max/Min pressure: {max_pressure.to(u.Pa)/min_pressure}")

        max_load, max_load_std = await self.get_load()  # Max load is with minumum pressure 

        print(f"Max load [N] (mean, std): [{max_load}, {max_load_std}]")
        
#         if max_load > optimum_load:
#             print(f"Max load larger then optimum. Operation complete.")
#             self.elevation = np.append(self.elevation, encoders.elevationCalculatedAngle)
#             self.pressure = np.append(self.pressure, min_pressure)
#             return 

        attcs.atpneumatics.cmd_m1SetPressure.set(pressure=max_pressure.to(u.Pa).value)
        self.attcs.atpneumatics.evt_m1SetPressure.flush()
        await attcs.atpneumatics.cmd_m1SetPressure.start()
        try:
            sp = await self.attcs.atpneumatics.evt_m1SetPressure.next(flush=False, timeout=120)
        except asyncio.TimeoutError:
            pass

        measured_pressure, measured_pressure_std = await self.get_pressure()

        min_load, min_load_std = await self.get_load()
        
        if min_load < self.min_allowed_load:
#             attcs.atpneumatics.cmd_m1SetPressure.set(pressure=0.1)
#             await attcs.atpneumatics.cmd_m1SetPressure.start()
            print(f"Min load {min_load} to small {self.min_allowed_load}. ")
#             return 
            
        print(f"Min load [Pa] (mean, std): ({min_load}, {min_load_std})")

        init_pressure = (min_pressure+max_pressure.to(u.Pa))/2.
        print(f"Init pressure: {init_pressure} ({init_pressure/max_pressure.to(u.Pa)})")

        attcs.atpneumatics.cmd_m1SetPressure.set(pressure=init_pressure.to(u.Pa).value)
        self.attcs.atpneumatics.evt_m1SetPressure.flush()
        await attcs.atpneumatics.cmd_m1SetPressure.start()
        try:
            sp = await self.attcs.atpneumatics.evt_m1SetPressure.next(flush=False, timeout=30)
        except asyncio.TimeoutError:
            pass

        niter = 0
        max_iter = 11
        success = True

        # You did this up there, why here too?
        init_pressure = (min_pressure+max_pressure.to(u.Pa))/2.
        print(f"Init pressure: {init_pressure} ({init_pressure/max_pressure.to(u.Pa)})")

        attcs.atpneumatics.cmd_m1SetPressure.set(pressure=init_pressure.to(u.Pa).value)
        self.attcs.atpneumatics.evt_m1SetPressure.flush()
        await attcs.atpneumatics.cmd_m1SetPressure.start()
        try:
            sp = await self.attcs.atpneumatics.evt_m1SetPressure.next(flush=False, timeout=30)
        except asyncio.TimeoutError:
            pass

        load, load_std = await self.get_load()
        
        print("****")
        print(f"load: {load}")
        print(f"Optimum load: {optimum_load}")
        delta = np.abs(load - optimum_load)/optimum_load
        print(f"{delta} > 0.1")
        print("****")

        
        while delta > self.tolerance:
            niter += 1
            print(f"Iter {niter}")
            print(f"Dif: {load-optimum_load}")

            if load - optimum_load < 0.:
                max_pressure = init_pressure
            else:
                min_pressure = init_pressure
            init_pressure = (min_pressure+max_pressure)/2.
            print(f"Init pressure: {init_pressure} ({init_pressure/max_pressure.to(u.Pa)})")

            attcs.atpneumatics.cmd_m1SetPressure.set(pressure=init_pressure.to(u.Pa).value)
            self.attcs.atpneumatics.evt_m1SetPressure.flush()
            await attcs.atpneumatics.cmd_m1SetPressure.start()
            try:
                sp = await self.attcs.atpneumatics.evt_m1SetPressure.next(flush=False, timeout=30)
            except asyncio.TimeoutError:
                pass
            
            measured_pressure, measured_pressure_std = await self.get_pressure()
            
            print(f"Set pressure: {init_pressure.to(u.Pa)}")
            print(f"Measured pressure [Pa] (mean, std): ({measured_pressure}, {measured_pressure_std})")
            print(f"Diff: {init_pressure.to(u.Pa).value-measured_pressure.to(u.Pa).value}")

            load, load_std = await self.get_load()

            print("****")
            print(f"load [Pa] (mean,std): ({load}, {load_std})")
            print(f"Optimum load [Pa]: {optimum_load}")
            delta = np.abs(load - optimum_load)/optimum_load
            print(f"diff: {delta} | tol: {self.tolerance}")
            print("****")
            
            

            if niter > max_iter:
                print("Failed")
                success = False
                break

        if success:
            print("Success!")
            optimum_pressure, optimum_pressure_std = await self.get_pressure()
            print(f"Optimum pressure: {optimum_pressure} @ el {encoders.elevationCalculatedAngle}")        
            self.elevation = np.append(self.elevation, encoders.elevationCalculatedAngle)
            self.pressure = np.append(self.pressure, optimum_pressure.to(u.Pa).value)    
            
        attcs.atpneumatics.cmd_m1SetPressure.set(pressure=0.1)
        await attcs.atpneumatics.cmd_m1SetPressure.start()

In [7]:
#attcs = ATTCS()

In [8]:
mp = MeasureM1Pressure(attcs)
mp.optimum_fraction = 1./3.

In [332]:
#235/24.
#353./3.
# Mass of mirror is 353.3 kg
mirror_mass = 353.3
#  HARDPOINT SHOULD READ 259 LBS
print(f"Mirror mass [lbs]: {mirror_mass*2.2/259.}")
print(f'Desired Load at zenith [kg]: {mirror_mass/24.0}')
print(f'Desired Load at zenith [lbs]: {mirror_mass/24.0*2.2}')
elevation=(attcs.atmcs.tel_mountEncoders.get()).elevationCalculatedAngle
adjusted_load=mirror_mass/24.0*np.sin(elevation*np.pi/180.)*3.
print(f'Desired Load at current elevation {elevation} is [kg]: {adjusted_load}')
print(f'Desired Load at current elevation {elevation} is [lbs]: {adjusted_load*2.2}')

# print(f'Desired Load at current elevation {elevation} is [kg]: {adjusted_load*2.5}')
# print(f'Desired Load at current elevation {elevation} is [lbs]: {adjusted_load*2.2*2.5}')

Mirror mass [lbs]: 3.0010038610038614
Desired Load at zenith [kg]: 14.720833333333333
Desired Load at zenith [lbs]: 32.38583333333334
Desired Load at current elevation 84.99999906597077 is [kg]: 43.99444829173049
Desired Load at current elevation 84.99999906597077 is [lbs]: 96.7877862418071


In [175]:
#THIS IS FOR TESTING - DELETE!!
#print(f'Desired Load at current elevation {elevation} is [lbs]: {28/32.*adjusted_load*2.2}')

In [307]:
#await mp.measure_optimum_pressure()

In [295]:
#await attcs.atpneumatics.cmd_m1OpenAirValve.start()
await attcs.atpneumatics.cmd_m1CloseAirValve.start()

CommandIdAck(cmd_id=1564448958, ack=(ack=303, error=0, result='Ok'))

In [293]:
attcs.atpneumatics.cmd_m1SetPressure.set(pressure=20.000) # 

True

In [294]:
await attcs.atpneumatics.cmd_m1SetPressure.start()

CommandIdAck(cmd_id=1564514594, ack=(ack=303, error=0, result='Ok'))

In [333]:
load, load_std = await mp.get_load(n=7)
tmp_mean, tmp_std = await mp.get_pressure(n=5)
elevation=(attcs.atmcs.tel_mountEncoders.get()).elevationCalculatedAngle
print(f'Elevation [deg]: {elevation}')
print(f'Pressure [Pa] (mean, std): {tmp_mean}, {tmp_std}')
print(f'Load [N] (mean, std): [{load}, {load_std}]')
print(f'Mass Supported [kg] (mean, std): [{load/9.81}, {load_std/9.81}]')

Elevation [deg]: 84.99999951302016
Pressure [Pa] (mean, std): 78291.54842773438 Pa, 514.704936759586 Pa
Load [N] (mean, std): [473.9323822631836, 7.064001274791568]
Mass Supported [kg] (mean, std): [48.31115007779649, 0.7200816793875197]


### set to 23000 and it went mental and back to 3.2 PSI!
# mirror lost air at pressure = 22100 but could be between 21900 and 22100
# dropped on 22100

#Elevation [deg]: 86.66942902978917
#Pressure [Pa] (mean, std): 22725.13008219401 Pa, 118.90673555913712 Pa
#Load [N]: 866.0481259643554
#Mass Supported [kg]: 88.37225775146483

In [ ]:
m1pressure= [10, 100, 1000, 2000, 4000, 6000, 8000, 12000,
            15000, 20000, 17500 ]
measured_pressure = [9469, 9570, 9584, 9389, 11444.9, 13638, 17205, 30170.2,
                    52330, 101140, 83395.8 ]
Load = [967.4, 966.4, 967.9, 971.4, 951.05, 939.37, 912.77, 815.72,
       641.98, 264, 392.8 ]

In [ ]:
m1pressure= [12000, 20000, 20500, 21000, 21300, 21600, 21900, 21950, 22000,]
measured_pressure = [30155, 100726, 109020, 113473, 114548, 116661, 122531, 122559, 123693]
Load = [811, 263.3, 192, 165, 154, 134, 9.1, 8.95, 7.95, ]

In [ ]:
await mp.measure_optimum_pressure()

In [ ]:
await mp.measure_optimum_pressure()

In [ ]:
await mp.measure_optimum_pressure()

In [ ]:
mp.elevation, mp.pressure

In [ ]:
data = np.array([mp.elevation, mp.pressure])

In [ ]:
data

In [ ]:
np.save("alt_pressure_2019-07-17.npy", data)

In [ ]:
fit = np.polyfit(mp.elevation, mp.pressure, 2)

In [ ]:
poly = np.poly1d(fit)
x = np.linspace(22., 90., 100)
y = poly(x)
plt.plot(mp.elevation, mp.pressure/1e3, 'o')
plt.plot(x, y/1e3, '-')
plt.xlabel("Altitude (deg)")
plt.ylabel("Pressure (kPa)")

In [ ]:
print(fit)

In [ ]:
tmp=np.array([1,2,3,4,5,6])
np.std(tmp)
np.mean(tmp)

In [ ]:
ss = attcs.atdome.evt_summaryState.get()
print(ss)

In [ ]:
salobj.State(ss.summaryState)

In [ ]:
await salobj.set_summary_state(attcs.atdome, salobj.State.ENABLED, settingsToApply='test')

In [ ]:
await salobj.set_summary_state(attcs.atdome, salobj.State.STANDBY)

In [ ]:
attcs.atdome.cmd_moveAzimuth.set(azimuth=0.)

In [ ]:
await attcs.atdome.cmd_moveAzimuth.start()

In [ ]:
await attcs.atdome.cmd_openShutter.start()

In [ ]:
await attcs.atdome.cmd_closeShutter.start()

In [ ]:
await mp.attcs.atpneumatics.evt_m

In [ ]:
sp = await mp.attcs.atpneumatics.evt_m1SetPressure.next(flush=False, timeout=5)

In [ ]:
sp.pressure

In [ ]:
await mp.attcs.atpneumatics.cmd_m1CloseAirValve.start()    

In [ ]:
mp.attcs.atpneumatics.cmd_m1SetPressure.set(pressure=0.1)
await mp.attcs.atpneumatics.cmd_m1SetPressure.start()

In [ ]:
pressure = 15.*u.imperial.psi
mp.attcs.atpneumatics.cmd_m1SetPressure.set(pressure=pressure.to(u.Pa).value)
await mp.attcs.atpneumatics.cmd_m1SetPressure.start()

In [ ]:
load = await mp.get_load()

In [ ]:
print(load)

In [ ]:
load2 = await mp.get_load()

In [ ]:
print(load2/3.)

In [ ]:
mp.min_allowed_load = load

In [ ]:
mp.min_allowed_load

## Get the optimum pressure for a specified load

Set the optimum load. Measure the minimum pressure and load and get maximum pressure to run a search for the best pressure. 

In [ ]:
optimum_pressure = await get_pressure(10, attcs.atpneumatics.tel_m1AirPressure)
print(f"Optimum pressure: {optimum_pressure}")

In [ ]:
for c in attcs.components():
    r = getattr(attcs, c.lower())
    await salobj.set_summary_state(r, salobj.State.STANDBY)